In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [3]:
#more imports; mine, not there by default on Kaggle Kernel
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression

from sklearn.tree import DecisionTreeClassifier

from sklearn.svm import SVC

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier


from sklearn.model_selection import validation_curve
from sklearn.model_selection import learning_curve
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.decomposition import PCA

from sklearn.neighbors import KNeighborsClassifier

from numbers import Number

In [4]:
df = pd.read_csv("./input/train.csv")
# print columns w nas, if any
print(df.columns[pd.isnull(df).any()].tolist())
#no nas

df[:3]

[]


,id,bone_length,rotting_flesh,hair_length,has_soul,color,type
0,0,0.354512,0.350839,0.465761,0.781142,clear,Ghoul
1,1,0.575560,0.425868,0.531401,0.439899,green,Goblin
2,2,0.467875,0.354330,0.811616,0.791225,black,Ghoul


In [5]:
#dummy out color
dums = pd.get_dummies(df['color'], prefix = "col", drop_first = True )
df = pd.concat([df,dums], axis=1)
df[:3]

,id,bone_length,rotting_flesh,hair_length,has_soul,color,type,col_blood,col_blue,col_clear,col_green,col_white
0,0,0.354512,0.350839,0.465761,0.781142,clear,Ghoul,0.0,0.0,1.0,0.0,0.0
1,1,0.575560,0.425868,0.531401,0.439899,green,Goblin,0.0,0.0,0.0,1.0,0.0
2,2,0.467875,0.354330,0.811616,0.791225,black,Ghoul,0.0,0.0,0.0,0.0,0.0


In [6]:
inputs = [col for col in df.columns if (col !='id' and col!= 'type' and col != 'color')]
y = df['type']

In [7]:
X = np.asarray(df[inputs])

In [36]:
#clf = AdaBoostClassifier()
#clf = GradientBoostingClassifier(learning_rate = .01)
#clf = VotingClassifier()



# #adaboost, gbc
# to_vary = 'n_estimators'
# varyrange = [10,50,100,200,300,500,800,1000,2000]
# #adaboost learn rate default: gradual increase in both scores w n; 
#     #going further there's a peak at 3500, validation accuracy ~ .69; much worse than SVC
# #gbc learn rate default: peak at 50;going further, 20=25=30 acc ~ .74
# #gbc learn rate .01: peak at 200, ~.7425
# varyrange = range(130,280,25)
# #gbc learn rate .003: peak at 300 and 500, ~.74

# clf = DecisionTreeClassifier(max_depth = 6)
# # to_vary = 'max_depth'
# # varyrange = [2,3,4,5,6,7,8,9,10,12,15]
# # #4 thru 8 about equal, around .67-.68
# to_vary = 'min_impurity_split'
# varyrange = np.arange(.03,.24,.01)
# #should run more to get better estimate, but .18 is usually a local optimum and near the global optimum (~.68)

# clf = KNeighborsClassifier()
# to_vary = 'n_neighbors'
# varyrange = [1,2,3,4,5,6,7,8,10,12,15,20]
# #best at 6 neighbors, score around .69

# clf = RandomForestClassifier(n_estimators = 50, max_depth = 6)
# to_vary = 'n_estimators'
# varyrange = [20,50,100,200]
# to_vary = 'min_impurity_split'
# varyrange = np.arange(.01,.24,.01)


train_scores, test_scores = validation_curve(clf, X, y, to_vary, varyrange)
# train_sizes, train_scores, test_scores = learning_curve(clf, X, y)


train_avg = np.mean(train_scores, axis=1)
test_avg = np.mean(test_scores, axis=1)

plt.clf()
plt.plot(varyrange, train_avg, color = 'k', marker = '.')
plt.plot(varyrange, test_avg, color = 'g', marker = '.')
# plt.plot(train_sizes, train_avg, color = 'k', marker = '.')
# plt.plot(train_sizes, test_avg, color = 'g', marker = '.')

plt.show()

In [69]:
# voting classifier
clf1 = SVC(kernel = 'sigmoid', C=232.5, probability = True)
clf2 = RandomForestClassifier(max_depth = 6, min_impurity_split = .18)
clf3 = GradientBoostingClassifier(learning_rate = .01, n_estimators = 200)
clf1.fit(X,y)
clf2.fit(X,y)
clf3.fit(X,y)

VotingClassifier(estimators=[('svm', SVC(C=232.5, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='sigmoid',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)), ('rf', RandomForestClassifier(bootstrap=True, cla...=200, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False))],
         n_jobs=1, voting='soft', weights=[2.5, 1.0, 1.0])

In [82]:
# vary0s = np.arange(.7,3.5,.1)
#max at 2.3, of around .755

vary0s = np.arange(2.1,2.52,.03)
#2.37 seems to do pretty well

vary_range = [[w,1,1] for w in vary0s]


test_avg = []
for wts in vary_range:
    test_scores = cross_val_score(VotingClassifier(estimators = [('svm',clf1),('rf',clf2),('gb',clf3)], weights = wts, voting = 'soft'), X, y)

    test_avg.append(np.mean(test_scores))
    

plt.clf()
plt.plot(vary0s, test_avg, color = 'g', marker = '.')
plt.show()

In [83]:
eclf = VotingClassifier(estimators = [('svm',clf1),('rf',clf2),('gb',clf3)], weights = [2.37,1.,1.], voting = 'soft')
eclf.fit(X,y)
eclf.score(X,y)

0.83557951482479786

In [84]:
dft = pd.read_csv("./input/test.csv")
#dummy out color
dums = pd.get_dummies(dft['color'], prefix = "col", drop_first = True )
dft = pd.concat([dft,dums], axis=1)
Xt = np.asarray(dft[inputs])

In [59]:
# yp = clf.predict(Xt)
# dfp = dft.copy()
# dfp['type'] = yp

In [85]:
#ensemble
yp = eclf.predict(Xt)
dfp = dft.copy()
dfp['type'] = yp

In [86]:
dfp.to_csv('softvoting4.csv', index = False, columns = ['id', 'type'])

In [87]:
dfp[:3]

,id,bone_length,rotting_flesh,hair_length,has_soul,color,col_blood,col_blue,col_clear,col_green,col_white,type
0,3,0.471774,0.387937,0.706087,0.698537,black,0.0,0.0,0.0,0.0,0.0,Ghoul
1,6,0.427332,0.645024,0.565558,0.451462,white,0.0,0.0,0.0,0.0,1.0,Goblin
2,9,0.549602,0.491931,0.660387,0.449809,black,0.0,0.0,0.0,0.0,0.0,Ghoul


In [44]:
eclf.score(X,y)

0.8571428571428571